In [1]:
!pip3 install setuptools
!pip3 install wheel
!pip3 install tensorflow
!pip3 install keras
!pip3 install pillow
!pip3 install sklearn
!pip3 install "numpy<1.17"

In [2]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab happy-or-sad.zip from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location
path = f"{getcwd()}/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [3]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            if(logs.get('acc') > 0.999):
                print("Reached 99.9% accuracy so cancelling training!")
                self.model.stop_training=True

    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model
    model = tf.keras.models.Sequential([
        # Apply 16 different filters of size 3x3, pre-defined by keras
        # 'relu' = ignore negative values
        # inputs are images with 300x300px and with true color channels, i.e. 3 bytes
        tf.keras.layers.Conv2D(16, (3,3), activation="relu", input_shape = (300,300,3)),
        # Pick thee highest of the 2x2px
        tf.keras.layers.MaxPooling2D(2,2),
        # Apply 32 different filters for size 3x3, pre-defined by keras
        # 'relu' = ignore negative values
        tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
        # Pick thee highest of the 2x2px
        tf.keras.layers.MaxPooling2D(2,2),
        # Apply 64 different filters for size 3x3, pre-defined by keras
        # 'relu' = ignore negative values
        tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
        # Pick thee highest of the 2x2px
        tf.keras.layers.MaxPooling2D(2,2),
        # Transofrm matrix into a single column
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        # Sigmoid is a good function for binary classifiers
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ])
        
    model.summary()

    from tensorflow.keras.optimizers import RMSprop

    model.compile(
        # tweakable learning rate
        optimizer=RMSprop(lr=0.001),
        loss='binary_crossentropy',
        metrics=['acc']
    )
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Rescale image colors from 0.0->1.0
    train_datagen = ImageDataGenerator(rescale=1./255)
    
    train_dir = os.path.join('/tmp/h-or-s/')
    train_happy_dir = os.path.join('/tmp/h-or-s/happy')
    train_sad_dir   = os.path.join('/tmp/h-or-s/sad')

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(300,300),
        batch_size=20,
        class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit_generator(
        train_generator,
        steps_per_epoch=5,
        epochs=15,
        verbose=1,
        callbacks=[callbacks]
    )
    # model fitting
    return history.history['acc'][-1]

In [4]:
# The Expected output: "Reached 99.9% accuracy so cancelling training!""
train_happy_sad_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
____

1.0